# **ECG TECHNICIAN ASSIGNMENT OPTIMIZATION MODEL**

In [12]:
# pip install pulp

In [23]:
import pandas as pd
import pulp
import numpy as np
from datetime import datetime

In [24]:
# Load the fault dataset
df_faults = pd.read_csv('../dataset/ecg_faults_dataset.csv')
df_faults.head()

,Fault_ID,Town,Zone,Distance_km,Travel_time_hours,Fault_type,Repair_time_hours,Priority
0,F1,Afadzo South,Far,38.4,1.017,Pole replacement,1.00,High
1,F2,Liati,Far,20.0,0.550,Transformer maintenance,0.75,Normal
2,F3,Golokwati,Far,21.3,0.500,Transformer maintenance,0.75,High
3,F4,Agome yo,Near,15.9,0.517,Network line extension,2.00,Normal
4,F5,Logba,Far,38.3,1.000,Network line extension,2.00,Normal


In [25]:
# summary
print("DATA SUMMARY")
print(f"Faults by Zone:")
print(df_faults['Zone'].value_counts())
print(f"\nFaults by Type:")
print(df_faults['Fault_type'].value_counts())
print(f"\nAverage travel time (Near): {df_faults[df_faults['Zone']=='Near']['Travel_time_hours'].mean():.3f} hours")
print(f"Average travel time (Far): {df_faults[df_faults['Zone']=='Far']['Travel_time_hours'].mean():.3f} hours")

DATA SUMMARY
Faults by Zone:
Zone
Near    8
Far     7
Name: count, dtype: int64

Faults by Type:
Fault_type
Transformer maintenance          5
Cable joining and termination    4
Network line extension           3
Transformer installation         2
Pole replacement                 1
Name: count, dtype: int64

Average travel time (Near): 0.469 hours
Average travel time (Far): 0.748 hours


#### **EXTRACTING PARAMETERS FROM FAULTS DATASET**

In [26]:
# Define problem dimensions
num_faults = len(df_faults)
num_groups = 5

# Create index sets
faults = list(range(num_faults))  # set J 
groups = list(range(num_groups))   # set I 

print(f"Sets defined:")
print(f"  I (groups): {groups} = {num_groups} groups")
print(f"  J (faults): {faults[0]} to {faults[-1]} = {num_faults} faults")

Sets defined:
  I (groups): [0, 1, 2, 3, 4] = 5 groups
  J (faults): 0 to 14 = 15 faults


In [27]:
# Extract parameters from data
travel_time = df_faults['Travel_time_hours'].tolist()  # t_j
repair_time = df_faults['Repair_time_hours'].tolist()  # r_j
zones = df_faults['Zone'].tolist()                      # z_j

print(f"\nParameters extracted:")
print(f"  t_j (travel times): {len(travel_time)} values")
print(f"  r_j (repair times): {len(repair_time)} values")
print(f"  z_j (zones): {len(zones)} values")


Parameters extracted:
  t_j (travel times): 15 values
  r_j (repair times): 15 values
  z_j (zones): 15 values


In [28]:
# Define other parameters
Q = 3           # Capacity per group
H = 8           # Shift duration in hours
theta = 1.7     # Equity tolerance (Far can be 1.7x Near)
alpha = 0.7     # Efficiency weight
beta = 0.3      # Equity weight

print(f"\nFixed parameters:")
print(f"  Q (capacity per group): {Q} faults")
print(f"  H (shift duration): {H} hrs")
print(f"  θ (equity tolerance): {theta}")
print(f"  α (efficiency weight): {alpha}")
print(f"  β (equity weight): {beta}")


Fixed parameters:
  Q (capacity per group): 3 faults
  H (shift duration): 8 hrs
  θ (equity tolerance): 1.7
  α (efficiency weight): 0.7
  β (equity weight): 0.3


In [29]:
# Identify zones indexes
near_faults = [j for j in faults if zones[j] == 'Near']
far_faults = [j for j in faults if zones[j] == 'Far']

print(f"\nZones:")
print(f"  Near faults: {near_faults}")
print(f"  Far faults: {far_faults}")


Zones:
  Near faults: [3, 6, 7, 8, 10, 12, 13, 14]
  Far faults: [0, 1, 2, 4, 5, 9, 11]


#### **CREATING OPTIMIZATION MODEL**

In [30]:
# Create the model
prob = pulp.LpProblem("ECG_Technician_Assignment", pulp.LpMinimize)
prob

ECG_Technician_Assignment:
MINIMIZE
None
VARIABLES

#### Define decision variables

In [31]:
# Decision variables: x[i,j] = 1 if group i assigned to fault j, 0 otherwise
x = pulp.LpVariable.dicts(
    "assign",
    [(i, j) for i in groups for j in faults],
    cat='Binary'
)

print(f"Created {len(x)} binary decision variables x_ij")
print(f"Format: x[i,j] where i ∈ {{{groups[0]},...,{groups[-1]}}} and j ∈ {{{faults[0]},...,{faults[-1]}}}")

Created 75 binary decision variables x_ij
Format: x[i,j] where i ∈ {0,...,4} and j ∈ {0,...,14}


#### Define objective function

In [32]:
# initial obj function
objective = pulp.lpSum([
    travel_time[j] * x[i, j]
    for i in groups
    for j in faults
])

prob += objective, "Total_Travel_Time"

print("Objective function: Minimize Total Travel Time")
print("Formula: Σ_i Σ_j (t_j × x_ij)")

Objective function: Minimize Total Travel Time
Formula: Σ_i Σ_j (t_j × x_ij)


#### Add Constraint 1 - **coverage**

In [33]:
# CONSTRAINT 1
# Σ_i x_ij = 1, ∀j ∈ J
constraint_count = 0
for j in faults:
    prob += (
        pulp.lpSum([x[i, j] for i in groups]) == 1,
        f"Coverage_Fault_{j}"
    )
    constraint_count += 1

print(f"Constraint 1 (Coverage): {constraint_count} constraints added")
print(f"Formula: Σ_i x_ij = 1, ∀j ∈ J")
print(f"Each fault assigned to exactly one group")

Constraint 1 (Coverage): 15 constraints added
Formula: Σ_i x_ij = 1, ∀j ∈ J
Each fault assigned to exactly one group


#### Add Constraint 2 - **capacity**

In [34]:
# CONSTRAINT 2
# Σ_j x_ij ≤ Q, ∀i ∈ I
for i in groups:
    prob += (
        pulp.lpSum([x[i, j] for j in faults]) <= Q,
        f"Capacity_Group_{i}"
    )
    constraint_count += 1

print(f"Constraint 2 (Capacity): {num_groups} constraints added")
print(f"Formula: Σ_j x_ij ≤ {Q}, ∀i ∈ I")
print(f"Each group assigned at most {Q} faults")

Constraint 2 (Capacity): 5 constraints added
Formula: Σ_j x_ij ≤ 3, ∀i ∈ I
Each group assigned at most 3 faults


#### Add Constraint 3 - **Time Feasibility**

In [35]:
# CONSTRAINT 3
# Σ_j (t_j + r_j) * x_ij ≤ H, ∀i ∈ I
for i in groups:
    prob += (
        pulp.lpSum([
            (travel_time[j] + repair_time[j]) * x[i, j]
            for j in faults
        ]) <= H,
        f"Time_Group_{i}"
    )
    constraint_count += 1

print(f"Constraint 3 (Time Feasibility): {num_groups} constraints added")
print(f"Formula: Σ_j (t_j + r_j) × x_ij ≤ {H}, ∀i ∈ I")
print(f"Total work time per group ≤ {H} hours")

Constraint 3 (Time Feasibility): 5 constraints added
Formula: Σ_j (t_j + r_j) × x_ij ≤ 8, ∀i ∈ I
Total work time per group ≤ 8 hours


#### Add Constraint 4 - Equity

In [36]:
# CONSTRAINT 4
# limiting maximum response time for far zone
# Average travel time to far zone ≤ theta × average to near zone

# Calculate expected average travel times if all faults are covered
if near_faults and far_faults:
    
    # Average travel time for near faults in the solution
    near_avg_travel = pulp.lpSum([travel_time[j] * pulp.lpSum([x[i, j] for i in groups]) 
                                   for j in near_faults]) / len(near_faults)
    
    # Average travel time for far faults in the solution  
    far_avg_travel = pulp.lpSum([travel_time[j] * pulp.lpSum([x[i, j] for i in groups]) 
                                  for j in far_faults]) / len(far_faults)
    
    # Equity constraint: far average ≤ theta × near average
    # Since Σ_i x[i,j] = 1 for all j (from coverage), this simplifies
    prob += (
        pulp.lpSum([travel_time[j] for j in far_faults]) / len(far_faults) 
        <= theta * pulp.lpSum([travel_time[j] for j in near_faults]) / len(near_faults),
        "Equity_Constraint"
    )
    constraint_count += 1
    
    print(f"Constraint 4 (Equity): 1 constraint added")
    print(f"Formula: Avg(t_j for j ∈ Far) ≤ {theta} × Avg(t_j for j ∈ Near)")
    print(f"Far zone avg travel time ≤ {theta}x Near zone avg")
else:
    print(f"Constraint 4 (Equity): SKIPPED (need both Near and Far faults)")

print(f"\nTotal constraints added: {constraint_count}")

Constraint 4 (Equity): 1 constraint added
Formula: Avg(t_j for j ∈ Far) ≤ 1.7 × Avg(t_j for j ∈ Near)
Far zone avg travel time ≤ 1.7x Near zone avg

Total constraints added: 26


#### **SOLVE THE MODEL**

In [37]:
# MILP model
start_time = datetime.now()
prob.solve(pulp.PULP_CBC_CMD(msg=1))  # CBC solver
end_time = datetime.now()
solve_time = (end_time - start_time).total_seconds()

print(f"\nSolver finished in {solve_time:.2f} seconds")

# Check solution status
status = pulp.LpStatus[prob.status]
print(f"\nSolution Status: {status}")

if status == 'Optimal':
    print("OPTIMAL SOLUTION FOUND!")
elif status == 'Feasible':
    print("Feasible solution found (may not be optimal)")
elif status == 'Infeasible':
    print("PROBLEM IS INFEASIBLE - No solution exists!")
    print("Possible reasons:")
    print(" - Too many faults for available capacity")
    print(" - Time constraints too tight")
    print(" - Equity constraint too restrictive")
elif status == 'Unbounded':
    print("PROBLEM IS UNBOUNDED")
else:
    print(f"Unexpected status: {status}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/kalamix/dsX/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/e988629093fa40d983491828560c2989-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/e988629093fa40d983491828560c2989-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 31 COLUMNS
At line 482 RHS
At line 509 BOUNDS
At line 585 ENDATA
Problem MODEL has 26 rows, 75 columns and 225 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 8.985 - 0.00 seconds
Cgl0005I 15 SOS with 75 members
Cgl0004I processed model has 25 rows, 75 columns (75 integer (75 of which binary)) and 225 elements
Cutoff increment increased from 1e-05 to 0.000999
Cbc0038I Initial state - 4 integers unsatisfied sum - 0.0739496
Cbc0038I Pass   1: suminf.    0.07066 (4) obj. 8.985 iterations 10
Cbc0038I Pass   2: suminf.    

#### **Extract and Display Results**

In [38]:
if status == 'Optimal' or status == 'Feasible':
    print("SOLUTION RESULTS")
    
    # Extract solution
    assignment_results = []
    
    for i in groups:
        assigned_faults_for_group = []
        total_travel = 0
        total_repair = 0
        
        for j in faults:
            if x[i, j].varValue == 1:  
                assigned_faults_for_group.append(j)
                total_travel += travel_time[j]
                total_repair += repair_time[j]
        
        if assigned_faults_for_group:  
            assignment_results.append({
                'Group': i,
                'Num_Faults': len(assigned_faults_for_group),
                'Assigned_Faults': assigned_faults_for_group,
                'Total_Travel_hrs': round(total_travel, 3),
                'Total_Repair_hrs': round(total_repair, 3),
                'Total_Time_hrs': round(total_travel + total_repair, 3)
            })
    
    # Display assignment summary
    print("\nASSIGNMENT SUMMARY BY GROUP:")
    for result in assignment_results:
        print(f"\nGroup {result['Group']}:")
        print(f"  Number of faults: {result['Num_Faults']}")
        print(f"  Assigned faults: {result['Assigned_Faults']}")
        print(f"  Total travel time: {result['Total_Travel_hrs']} hours")
        print(f"  Total repair time: {result['Total_Repair_hrs']} hours")
        print(f"  Total work time: {result['Total_Time_hrs']} hours (out of {H} available)")
    
    # Display detailed assignment
    print()
    print("DETAILED FAULT ASSIGNMENTS:")
    
    detailed_assignments = []
    
    for i in groups:
        for j in faults:
            if x[i, j].varValue == 1:
                detailed_assignments.append({
                    'Group': i,
                    'Fault_ID': df_faults.loc[j, 'Fault_ID'],
                    'Town': df_faults.loc[j, 'Town'],
                    'Zone': zones[j],
                    'Fault_Type': df_faults.loc[j, 'Fault_type'],
                    'Travel_hrs': travel_time[j],
                    'Repair_hrs': repair_time[j],
                    'Total_hrs': travel_time[j] + repair_time[j]
                })
    
    df_assignments = pd.DataFrame(detailed_assignments)
    df_assignments = df_assignments.sort_values(['Group', 'Town'])
    print(df_assignments.to_string(index=False))
    
    # Calculate overall metrics
    print()
    print("OVERALL PERFORMANCE METRICS:")
    
    total_travel_all = sum([result['Total_Travel_hrs'] for result in assignment_results])
    total_repair_all = sum([result['Total_Repair_hrs'] for result in assignment_results])
    total_time_all = total_travel_all + total_repair_all
    
    print(f"\nTotal travel time (all groups): {total_travel_all:.3f} hours")
    print(f"Total repair time (all groups): {total_repair_all:.3f} hours")
    print(f"Total work time (all groups): {total_time_all:.3f} hours")
    print(f"Average time per group: {total_time_all/num_groups:.3f} hours")
    print(f"Objective function value: {pulp.value(prob.objective):.3f}")
    
    # Zone equity analysis
    print()
    print("ZONE EQUITY ANALYSIS:")
    
    near_assignments = df_assignments[df_assignments['Zone'] == 'Near']
    far_assignments = df_assignments[df_assignments['Zone'] == 'Far']
    
    if len(near_assignments) > 0 and len(far_assignments) > 0:
        avg_travel_near = near_assignments['Travel_hrs'].mean()
        avg_travel_far = far_assignments['Travel_hrs'].mean()
        equity_ratio = avg_travel_far / avg_travel_near
        
        print(f"\nNear zone ({len(near_assignments)} faults):")
        print(f"  Average travel time: {avg_travel_near:.3f} hours ({avg_travel_near*60:.1f} min)")
        
        print(f"\nFar zone ({len(far_assignments)} faults):")
        print(f"  Average travel time: {avg_travel_far:.3f} hours ({avg_travel_far*60:.1f} min)")
        
        print(f"\nEquity ratio (Far/Near): {equity_ratio:.2f}")
        print(f"Equity constraint limit: {theta}")
        
        if equity_ratio <= theta:
            print(f"Equity constraint SATISFIED ({equity_ratio:.2f} ≤ {theta})")
        else:
            print(f"Equity constraint VIOLATED ({equity_ratio:.2f} > {theta})")
    
    # Save results
    df_assignments.to_csv('../result/optimization_results.csv', index=False)
    print("Results saved.")
else:
    print("\nNo solution to display (problem is infeasible)")

SOLUTION RESULTS

ASSIGNMENT SUMMARY BY GROUP:

Group 0:
  Number of faults: 3
  Assigned faults: [2, 9, 12]
  Total travel time: 1.566 hours
  Total repair time: 5.08 hours
  Total work time: 6.646 hours (out of 8 available)

Group 1:
  Number of faults: 3
  Assigned faults: [0, 1, 3]
  Total travel time: 2.084 hours
  Total repair time: 3.75 hours
  Total work time: 5.834 hours (out of 8 available)

Group 2:
  Number of faults: 3
  Assigned faults: [7, 8, 13]
  Total travel time: 1.367 hours
  Total repair time: 3.08 hours
  Total work time: 4.447 hours (out of 8 available)

Group 3:
  Number of faults: 3
  Assigned faults: [4, 10, 14]
  Total travel time: 1.867 hours
  Total repair time: 3.5 hours
  Total work time: 5.367 hours (out of 8 available)

Group 4:
  Number of faults: 3
  Assigned faults: [5, 6, 11]
  Total travel time: 2.101 hours
  Total repair time: 4.66 hours
  Total work time: 6.761 hours (out of 8 available)

DETAILED FAULT ASSIGNMENTS:
 Group Fault_ID         Town Z